# Noise label MMSE
https://www.overleaf.com/11871051tmqzsmsbdxrk#/44989926/

In [1]:
%matplotlib inline
import math
import os
import data_util
import BMapModel
#from data_util import DataPoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import faiss
import util
import scipy
# import joblib # version incompatibel with sklearn's joblib and can't load the previous model

from scipy.sparse import save_npz, load_npz
from sklearn.externals import joblib # store classifiers
from sklearn.preprocessing import MultiLabelBinarizer # convert y to {0,1}^L
from sklearn.preprocessing import StandardScaler # normalize features 
from sklearn.feature_extraction import DictVectorizer # extract feature vector to x
from numpy.random import normal # generate transforming matrix
from sklearn.neighbors import KDTree #KDTree for fast kNN search
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import average_precision_score
from joblib import Parallel, delayed # Multitread
from pytictoc import TicToc
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

Failed to load GPU Faiss: No module named swigfaiss_gpu
Faiss falling back to CPU-only.


In [2]:
!ls -R ../data

../data:
AmazonCat	   Bibtex	   Eurlex     README_Datasets
AmazonCat-14K	   Delicious	   Mediamill  Wiki10
AmazonCat-14K.zip  DeliciousLarge  RCV1-x     XMLDatasetRead

../data/AmazonCat:
amazonCat_test.txt  amazonCat_train.txt  X_te.npz  X_tr.npz  Y_te.npz  Y_tr.npz

../data/AmazonCat-14K:
amazonCat-14K_test.txt	 X_te.npz  Y_te.npz
amazonCat-14K_train.txt  X_tr.npz  Y_tr.npz

../data/Bibtex:
Bibtex_data.txt     bibtex_tstSplit.txt  X_tr.npz  Y_tr.npz
bibtex_trSplit.txt  X_te.npz		 Y_te.npz

../data/Delicious:
Delicious_data.txt	X_te.npz  X_tr.pkl  Y_tr.npz
delicious_trSplit.txt	X_te.pkl  Y_te.npz  Y_tr.pkl
delicious_tstSplit.txt	X_tr.npz  Y_te.pkl

../data/DeliciousLarge:
deliciousLarge_test.txt   X_te.npz  Y_te.npz
deliciousLarge_train.txt  X_tr.npz  Y_tr.npz

../data/Eurlex:
eurlex_test.txt  eurlex_train.txt  X_te.npz  X_tr.npz  Y_te.npz  Y_tr.npz

../data/Mediamill:
Mediamill_data.txt  mediamill_trSplit.txt  mediamill_tstSplit.txt

../data/RCV1-x:
rcv1x_test.txt	 X_te.npz  X_tr.npz

In [3]:
ls ../data/Delicious/Delicious_data.txt

../data/Delicious/Delicious_data.txt*


In [4]:
data_dir = "../data"
model_dir = "../noise_model/model1"
path = '/Delicious'

data_path = data_dir+path
clean_model_path = model_dir + path +'/clean'
noise_model_path = model_dir + path +'/noise'
num_core = -1
L_hat = 100
time = TicToc()

In [5]:
[X_tr, X_te, Y_tr, Y_te] = [load_npz(os.path.join(data_path, '{}.npz'.format(name)))\
                            for name in ['X_tr', 'X_te', 'Y_tr', 'Y_te']]

In [6]:
X_tr.shape, X_te.shape, Y_tr.shape, Y_te.shape

((12920, 500), (3185, 500), (12920, 983), (3185, 983))

In [7]:
# use np array for simplicity
[X_tr, X_te, Y_tr, Y_te] = [x.toarray() for x in [X_tr, X_te, Y_tr, Y_te]]

## Add Noise to Label
We can assume that $\tilde{y}_i$ is generated by flipping the elements in $y^i$ with some probability. In particular, we assume that when $y^i_{j}=0$, $P({\tilde{y}^i_{j}=1\mid y^i_{j}=0}) = \rho_+$, and when $y^i_{j}=1$, $P({\tilde{y}^i_{j}=0\mid y^i_{j}=1}) = \rho_-$. We assume the label flip probabilities are the same across all the data points and classes.

Lets assume $\rho+=0.01, \rho-=0.5$ for example.



In [8]:
def flip_bits(message, p0, p1):
    '''
    randomly flip every "0->1" w/ prob p1, and every "1->0" w/ p0
    '''
    def flip(bit):
        if bit==1 and np.random.rand()<p0:
            bit = 0
        if bit==0 and np.random.rand()<p1:
            bit=1
        return bit
    np.random.seed(0)
    return np.apply_along_axis(lambda bits: np.array([flip(bit) for bit in bits]), 0, message)

In [9]:
RHO0, RHO1=0.3, 0.1
Y_tr_flip = flip_bits(Y_tr, RHO0, RHO1)

## Recover Label $\hat y$ from the noise label using MMSE

$$\hat{y}_j = E(y_j\mid \tilde{y}_j) = \frac{P(y_j=1)P(\tilde{y}_j\mid y_j=1)}{P(\tilde{y}_j)} = \begin{cases}
\frac{P(y_j=1)}{P(\tilde{y}_j=1)} (1-\rho_-) & \text{ if } \tilde{y}_j = 1 \\
\frac{P(y_j=1)}{P(\tilde{y}_j=0)} \rho_- & \text{ if } \tilde{y}_j = 0
\end{cases}$$

Let $$\hat y_1=\frac{P(y_j=1)}{P(\tilde{y}_j=1)} (1-\rho_-)$$
$$\hat y_2=\frac{P(y_j=1)}{P(\tilde{y}_j=0)} \rho_-$$

In [10]:
def MMSE(Y_flip, p0, p1):
    prob1_hat = np.sum(Y_flip)/float(Y_flip.shape[0]*Y_flip.shape[1])
    prob1 = (prob1_hat-p1)/(1-p1-p0)
    y_hat_1=prob1/prob1_hat*(1-p0)
    y_hat_0 = prob1/(1-prob1_hat)*p0
    return np.apply_along_axis(lambda bits: np.array([y_hat_1 if bit==1 else y_hat_0 for bit in bits]), 
                               0, Y_flip)

In [11]:
Y_tr_mmse = MMSE(Y_tr_flip, RHO0, RHO1)

In [12]:
[Z_tr, Z_tr_flip, Z_tr_mmse] = [util.map_2_z(Y, L_hat) for Y in [Y_tr, Y_tr_flip, Y_tr_mmse]]

In [13]:
(Z_tr==Z_tr_flip).sum()/float(Z_tr.shape[0]*Z_tr.shape[1])

0.59525464396284833

In [14]:
(Z_tr==Z_tr_mmse).sum()/float(Z_tr.shape[0]*Z_tr.shape[1])

0.59572600619195049

In [15]:
(Z_tr_flip==Z_tr_mmse).sum()/float(Z_tr.shape[0]*Z_tr.shape[1])

0.95091408668730648

### Step 2: Train Model

#### 2.1 train binary classifiers on each bit

In [ ]:
def train_bit(bit):
    print "Trianning model for the {}th bit\n... ... ... \n".format(bit)
    #clf = LogisticRegression(solver='sag')
    clf = LinearSVC(dual=False)
    clf.fit(y=Z_tr[:, bit], X=X_tr)
    joblib.dump(clf, os.path.join(model_path , 'label{}.pkl'.format(bit)))
    print "{}th bit's model successfully stored in {}/label{}.pkl\n".format(bit, model_path, bit)

In [ ]:
from joblib import Parallel, delayed # Multitread
#Parallel(n_jobs=num_core)(delayed(train_bit)(i) for i in range(Z_tr.shape[1]))

#### 2.2 Store the lower degree space info for kNN

We use opensource faiss library from FAIR to speedup the ANN(Approximate Nearest Neighbor) search.

When dimension and data size is relatively small, we use the brute force kNN search.

In [ ]:
# faiss brute force search
nn_index = faiss.index_factory(Z_tr.shape[1], "Flat", faiss.METRIC_L2)   # build the index
nn_index.add(Z_tr.astype('float32'))

```Python
# index created by index factory
nn_index = faiss.index_factory(Z_tr.shape[1], "IVF100,Flat", faiss.METRIC_L2) # need train
nn_index.train(Z_tr.astype('float32'))
nn_index.add(Z_tr.astype('float32'))

print nn_index.nlist # number of clusters, only INF has this
nn_index.nprobe = 1 # number of clusters to search through, only INF has this, need to be validate
```

### Step 3 Prediction and Validation

In [ ]:
model = BMapModel.BM_Predictor(Y_tr.shape[1], L_hat, index=nn_index, Y_tr=Y_tr, model_path=model_path)

In [ ]:
# k=1 without voting
time.tic()
Y_pred = model.predict_y(X_te, 20, vote=40) # 1 nearest neighbor
time.toc()

In [ ]:
#average_precision_score(y_true=Y_te, y_score=Y_pred, average='weighted')

In [ ]:
util.precision_at_k(Y_te, Y_pred, 1)

In [ ]:
def validate_voter(voter):
    Y_pred = model.predict_y(X_te, vote=voter, weighted=True)
    return (util.precision_at_k(Y_te, Y_pred, 1))

In [ ]:
p_at_k_votes = Parallel(n_jobs=num_core)\
                    (delayed(validate_voter)(voter) for voter in range(1, 100))

In [ ]:
plt.plot(range(1,100), p_at_k_votes)
plt.xlabel('number of voters in kNN')
plt.ylabel('p@1 score')
plt.title('OvsA, L_hat={}, {}'.format(L_hat, train_filename))
top = np.argmax(p_at_k_votes)
p_at_k_votes[top], top

#### 3.2 See the model performance under no error channel
Given our predicted value is the correct "Z_te", what performance can our model achieve?

In [ ]:
def validate_model(L_hat, Y_tr, Y_te, pk=1, vote=10):
    Z_tr = util.map_2_z(Y_tr, L_hat)
    z_te = util.map_2_z(Y_te, L_hat)
    # faiss brute force search
    knn_index = faiss.index_factory(Z_tr.shape[1], "Flat", faiss.METRIC_L2)   # build the index
    knn_index.add(Z_tr.astype('float32'))

    model = BMapModel.BM_Predictor(Y_tr.shape[1], L_hat, index=knn_index, Y_tr=Y_tr)

    y_pred_fake = model.vote_y(z_te, vote=vote, weighted=False)
    return util.precision_at_k(Y_te, y_pred_fake, pk)

In [ ]:
pk=1;vote=40
L_hat_free_score = Parallel(n_jobs=num_core)\
                    (delayed(validate_model)(L_hat, Y_tr, Y_te, pk, vote) for L_hat in range(1, 120))

In [ ]:
plt.plot(range(1,120), L_hat_free_score)
plt.xlabel('L_hat')
plt.ylabel('p@1 score')
plt.title('Eurlex: The model capacity for Bmap and kNN')

#### 3.3 optimize hyperparameter
use  k fold cross validation to optimize over 

In [ ]:
# validate the result with different L_hat under the same model
def validate(L_hat, pk=1, vote=20): # simple forkable parallel for loop body
    from util import map_2_z
    from util import precision_at_k
    #k_fold = KFold(n_splits=fold)
    #print "L_hat is now {}\n".format(L_hat)
    p_sum = 0
   # for train_index, test_index in k_fold.split(X_tr):
    x_train = X_tr
    y_train = Y_tr
    x_test = X_te
    y_test = Y_te

    # map and create kNN index
    z_train = map_2_z(y_train, L_hat)
    # faiss brute force search
    knn_index = faiss.index_factory(z_train.shape[1], "Flat", faiss.METRIC_L2)   # build the index
    knn_index.add(z_train.astype('float32'))

    # construct model
    model = BMapModel.BM_Predictor(Y_tr.shape[1], L_hat, index=knn_index, Y_tr=y_train)
    model.load_clf(model_path)
    #predict and calculate p@k score
    y_pred = model.predict_y(x_test, vote=vote, weighted=True)
    # precision@pk
    #p_sum += precision_at_k(y_test, y_pred, k=pk)
    return precision_at_k(y_test, y_pred, k=pk)


In [ ]:
# Optimize L_hat's value on the metric precision@k
pk=1
vote=40
L_hat_range = range(1, 200)

In [ ]:
L_hat_score = Parallel(n_jobs=num_core)(delayed(validate)(L_hat, pk, vote) for L_hat in L_hat_range)

In [ ]:
line_up, = plt.plot(range(1,200), L_hat_free_score, label='Model Capacity')
line_down, = plt.plot(range(1,200), L_hat_score, label='OvsA performance')
plt.legend(handles=[line_up, line_down])
plt.xlabel('L_hat')
plt.ylabel('precision@{}'.format(pk))
plt.title('Delicious validation on L_hat')

### 3.4 Bit Flip Probability
the classifiers predict $\hat z$ can be viewed as transmiting z from a BSC channel with some bit flip probability, this is actually representing the prediction accuracy.

In [ ]:
L_hat, X_te.shape[0]

In [ ]:
def validate_channel(X_te, Y_te):
    z_te = util.map_2_z(Y_te, L_hat)
    # use the classifers to predict z_hat
    model = BMapModel.BM_Predictor(Y_tr.shape[1], L_hat, model_path=model_path)
    z_pred = model.predict_z(X_te)
    
    hamming = []
    for i in range(z_te.shape[0]):
        hamming.append((z_pred[i]!=z_te[i]).sum())
    return np.array(hamming) / float(z_te.shape[1])

In [ ]:
test_error = validate_channel(X_te, Y_te)

In [ ]:
sns.distplot(test_error, bins=20)
plt.xlabel('error rate between z_te and z_pred')
plt.ylabel('density')
plt.title('Delicious: distribution of bit_flip')

In [ ]:
test_error.mean()

In [ ]:
training_error = validate_channel(X_tr, Y_tr)

In [ ]:
sns.distplot(training_error, bins=20)
plt.xlabel('error rate between z_train and z_pred')
plt.ylabel('density')
plt.title('Delicious: distribution of training error')

In [ ]:
training_error.mean()

### 3.5 Train and test model directly on the X and Y 

In [ ]:
model_dir_mirror = model_dir+"_origin"
model_path_mirror = model_dir_mirror+path